## Importing the Required Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV

In [2]:
heart_data = pd.read_csv('heart_disease_data.csv')
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
heart_data['cp'].value_counts()

0    143
2     87
1     50
3     23
Name: cp, dtype: int64

In [4]:
#checking the data for missing values
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [5]:
heart_data.shape


(303, 14)

In [6]:
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [7]:
# checking the distribution of our target
heart_data['target'].value_counts()

1    165
0    138
Name: target, dtype: int64

1 ----> heart disease
0 ----> No heart disease

In [8]:
# splitting data into features and target

X = heart_data.drop(columns='target', axis = 1)
y = heart_data['target']

## splitting data into Train/Test

In [9]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)
X[0:5]

array([[ 0.9521966 ,  0.68100522,  1.97312292,  0.76395577, -0.25633371,
         2.394438  , -1.00583187,  0.01544279, -0.69663055,  1.08733806,
        -2.27457861, -0.71442887, -2.14887271],
       [-1.91531289,  0.68100522,  1.00257707, -0.09273778,  0.07219949,
        -0.41763453,  0.89896224,  1.63347147, -0.69663055,  2.12257273,
        -2.27457861, -0.71442887, -0.51292188],
       [-1.47415758, -1.46841752,  0.03203122, -0.09273778, -0.81677269,
        -0.41763453, -1.00583187,  0.97751389, -0.69663055,  0.31091206,
         0.97635214, -0.71442887, -0.51292188],
       [ 0.18017482,  0.68100522,  0.03203122, -0.66386682, -0.19835726,
        -0.41763453,  0.89896224,  1.23989692, -0.69663055, -0.20670527,
         0.97635214, -0.71442887, -0.51292188],
       [ 0.29046364, -1.46841752, -0.93851463, -0.66386682,  2.08204965,
        -0.41763453,  0.89896224,  0.58393935,  1.43548113, -0.37924438,
         0.97635214, -0.71442887, -0.51292188]])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state = 2 )

In [11]:
print(X.shape, X_train.shape, X_test.shape, y_test.shape)

(303, 13) (242, 13) (61, 13) (61,)


# Model Training

used GridSearchCV to pick the best hyperparameters for the logistic Regression model

In [12]:
param = {'C' : [0.01, 0.1, 1], 'solver' : ['lbfgs', 'saga', 'liblinear', 'newton-cg', 'sag'], 'penalty' :['l2']}
lr = LogisticRegression()

model = GridSearchCV(lr, param, cv = 10)
model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1], 'penalty': ['l2'],
                         'solver': ['lbfgs', 'saga', 'liblinear', 'newton-cg',
                                    'sag']})

In [13]:
print ('the best hyperparamter is : ', model.best_params_)
print('accuracy:', model.best_score_)

the best hyperparamter is :  {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
accuracy: 0.8388333333333333


### Accuracy on the Training data

In [14]:
predicted_train_model = model.predict(X_train)
print('The predicted train model accuracy is : ', accuracy_score(predicted_train_model, y_train))

The predicted train model accuracy is :  0.8388429752066116


### Accuracy on the Test data

In [15]:
predicted_test_model = model.predict(X_test)
print('The predicted test model accuracy is : ' ,accuracy_score(predicted_test_model, y_test))

The predicted test model accuracy is :  0.819672131147541


calculating the log loss

In [16]:
y_proba = model.predict_proba(X_test)
log_loss(y_test, y_proba)

0.4824324740164179

Testing the accuracy of the model with new data

The model predicted well

In [17]:
import pickle
file = open ('log_regression.pkl', 'wb')
pickle.dump(model,file)
file.close()